In [1]:
include(joinpath("module", "lattice.jl"));

In [2]:
using BenchmarkTools
using .LatticeNetwork
using LinearAlgebra
using Random
using Profile
using ProfileView
using StatProfilerHTML
using Printf;

In [18]:
function short_dual_df(K::Int64, S_bar::Float64, E::Float64, T_power::NTuple{810000, Float64},
                    alpha_S_R::Float64, alpha_S_W::Float64, alpha_L_R::Float64, alpha_L_W::Float64, 
                    beta_S_R::Float64, beta_S_W::Float64, beta_L_R::Float64, beta_L_W::Float64,
                    power_S_H_R::Float64, power_S_H_W::Float64, power_L_H_R::Float64, power_L_H_W::Float64, power_S_F_R::Float64, power_S_F_W::Float64, power_L_F_R::Float64, power_L_F_W::Float64,
                    dR::Vector{Float64}, dW::Vector{Float64}, RW::Vector{Float64}, m::Vector{Float64}, n::Matrix{Float64})

    R = @view RW[1:K]
    W = @view RW[(K + 1):end]
    
    S_H_R = (alpha_S_R * (R .^ -power_S_H_R))
    S_H_W = W .^ -power_S_H_W
    
    L_H_R = R .^ -power_L_H_R
    L_H_W = (alpha_L_W * (W .^ -power_L_H_W))

    S_F = (beta_S_R * (R .^ -power_S_F_R)) .* (beta_S_W * (W .^ -power_S_F_W))
    L_F = (beta_L_R * (R .^ -power_L_F_R)) .* (beta_L_W * (W .^ -power_L_F_W))

    for i in 1:K
        Sn = sum(@views T_power[i, :] .* S_H_W .* n[i, :])  # ベクトル演算
        dR[i] = S_bar - S_H_R[i] * alpha_S_W * Sn - S_F[i] * m[i]
    end

    for j in 1:K
        Ln = sum(@views T_power[:, j] .* L_H_R .* n[:, j])  # ベクトル演算
        sum_n = sum(@views n[:, j])
        dW[j] = E * sum_n - L_H_W[j] * alpha_L_R * Ln - L_F[j] * m[j]
    end
    
    T_power = nothing

    return vcat(dR, dW)

end;

In [19]:
# 初期パラメータ設定
E = 5.0
Col = 30
K = Col * Col
M = 1.0
N = 1.0
Scaling = 10.0 / Col
alter_T_num = 0.5

# 格子ネットワークを生成
city_network = make_lattice(Col)

# 通勤費用パラメータ
t = 0.1
# 距離抵抗パラメータ
tau = 0.5
# 総土地供給量と平均床面積
S_total = 100
S_bar = S_total / K

# 座標データの生成
Coordinate_Data = [ 
    (city_network["node_dic"][string(i)]["x_pos"] * Scaling,
     city_network["node_dic"][string(i)]["y_pos"] * Scaling) 
    for i in 0:(K - 1)
]

# 距離行列の作成
distance_matrix = [sqrt(sum((Coordinate_Data[i] .- Coordinate_Data[j]).^2))
                   for i in 1:K, j in 1:K]

# モデルパラメータ設定
theta_firm = 1.0
theta_house = 1.0
alpha_1 = 0.4
alpha_2 = 0.4
beta_1 = 0.4
beta_2 = 0.4
# L = 1.0
# eta = 5.0
L = 0.2
eta = 1.2
p_proj = 1e-3
RW_ini = 1.0
RW_proj = 1e-3
err_short = 1e-5
err_long = 1e-3
obj_corr = 1.0
m_per = M / K
m0 = fill(m_per, K)
n0 = fill(N / (K * K), (K, K))
method = "FISTA"
dic = "Scaling=30"

K = Col * Col
T = max.(Scaling * t * alter_T_num, t * distance_matrix)
D = exp.(-tau * distance_matrix)

power_S_H_R = (1 - alpha_2) / (1 - alpha_1 - alpha_2)
power_S_H_W = alpha_2 / (1 - alpha_1 - alpha_2)
power_L_H_R = alpha_1 / (1 - alpha_1 - alpha_2)
power_L_H_W = (1 - alpha_1) / (1 - alpha_1 - alpha_2)

power_S_F_R = (1 - beta_2) / (1 - beta_1 - beta_2)
power_S_F_W = beta_2 / (1 - beta_1 -beta_2)
power_L_F_R = beta_1 / (1 - beta_1 - beta_2)
power_L_F_W = (1 - beta_1) / (1 - beta_1 - beta_2)

T_power = (1 ./ exp.(T)) .^ (1 / (1 - alpha_1 - alpha_2))
T_power = Tuple(T_power)

alpha_S_R = alpha_1 ^ power_S_H_R
alpha_S_W = alpha_2 ^ power_S_H_W
alpha_L_R = alpha_1 ^ power_L_H_R
alpha_L_W = alpha_2 ^ power_L_H_W

beta_S_R = beta_1 ^ power_S_F_R
beta_S_W = beta_2 ^ power_S_F_W
beta_L_R = beta_1 ^ power_L_F_R
beta_L_W = beta_2 ^ power_L_F_W

one_K = ones(K)
one_2K = ones(2 * K)

R_ini_list = ones(K)
W_ini_list = ones(K)
RW_ini_list = ones(2 * K)

short_itr = 10
long_itr = 1000;

In [20]:
dR = zeros(K)
dW = zeros(K)

for i in 1:20
    println("=======================================")
    println("Before allocation: ", Base.gc_bytes(), " bytes")
    short_dual_df(K, S_bar, E, T_power,
                    alpha_S_R, alpha_S_W, alpha_L_R, alpha_L_W, 
                    beta_S_R, beta_S_W, beta_L_R, beta_L_W,
                    power_S_H_R, power_S_H_W, power_L_H_R, power_L_H_W, power_S_F_R, power_S_F_W, power_L_F_R, power_L_F_W,
                    dR, dW, RW_ini_list, m0, n0)
    println("After allocation: ", Base.gc_bytes(), " bytes")

    GC.gc()
    println("After GC: ", Base.gc_bytes(), " bytes")
end

Before allocation: 5753684784 bytes


LoadError: MethodError: no method matching getindex(::NTuple{810000, Float64}, ::Int64, ::Colon)

[0mClosest candidates are:
[0m  getindex(::Tuple, ::Int64)
[0m[90m   @[39m [90mBase[39m [90m[4mtuple.jl:31[24m[39m
[0m  getindex(::Tuple, ::Integer)
[0m[90m   @[39m [90mBase[39m [90m[4mtuple.jl:32[24m[39m
[0m  getindex(::Tuple, ::Real)
[0m[90m   @[39m [90mBase[39m [90m[4mdeprecated.jl:103[24m[39m
[0m  ...
